# Aplicaciones de Ciencias de la Computación (INF265)
## Laboratorio 6: Agente aspirador de 6 posiciones

En ese laboratorio se implementa el entorno del aspirador de 6 posiciones y un agente reactivo para interactuar en el. 

    ########
    # A  B #
    # C  D # 
    # E  F #
    ########

Al final del notebook deberas responder a las preguntas planteadas. 

# Clase <b>Thing</b>

Esta clase generica representa cualquier objeto fisico que puede aparecer en un <b>Ambiente</b>. (No editar)  

In [ ]:
class Thing(object):

    def is_alive(self):
        """Cosas 'vivas'deben retornar true."""
        return hasattr(self, 'alive') and self.alive

    def show_state(self):
        """Muestra el estado interno del agente. Subclases deben sobreescribir esto."""
        print("I don't know how to show_state.")

# Clase <b>Agent</b>

Un agente es una subclase de Thing con un slot obligatorio: <b>.program</b>, el cual almacena la funcion que implementa el <b>programa del agente</b>. Esta funcion debe tomar como argumento la <b>percepcion</b> del agente y debe retornar una <b>accion</b>. La definicion de Percepcion y Accion depende del ambiente de trabajo (environment) donde el agente existe. El agente tambien puede tener el slot <b>.performance</b>, que mide el desempeño del agente en su ambiente.

In [ ]:
import collections
import random

class Agent(Thing):

    def __init__(self, program=None):
        self.alive = True
        self.performance = 0
        assert isinstance(program, collections.Callable)
        self.program = program

# Clase <b>Environment</b>

Esta clase abstracta representa un entorno de tareas. Clases de entornos reales heredan de esta. En un entorno tipicamente se necesitará implementar 2 cosas:
<b>percept</b>, que define la percepción que el agente ve; y 
<b>execute_action</b>, que define los efectos de ejecutar una acción. 
El entorno mantiene una lista de .things y .agents (el cual es un subconjunto de .things). Cada elemento de .things tiene un slot .location. (No editar)

In [ ]:
class Environment(object):

    def __init__(self):
        self.things = []
        self.agents = []

    def thing_classes(self):
        return []  # List of classes that can go into environment

    def percept(self, agent):
        """Retorna la percepción que el agente 'agent' ve en este punto."""
        raise NotImplementedError

    def execute_action(self, agent, action):
        """El agente 'agent' ejecuta una acción 'action' en el entorno."""
        raise NotImplementedError

    def default_location(self, thing):
        """Localización por defecto para colocar una nueva cosa sin localización especificada."""
        return None

    def is_done(self):
        """Retorna True si no hay ningón agente vivo"""
        return not any(agent.is_alive() for agent in self.agents)

    def add_thing(self, thing, location=None):
        """Añade una cosa thing al entorno en la localización location. 
           Si thing es un programa de agente, crea un nuevo agente con ese programa."""
        if not isinstance(thing, Thing):
            thing = Agent(thing)
        assert thing not in self.things, "No añade la misma cosa dos veces"
        thing.location = location if location is not None else self.default_location(thing)
        self.things.append(thing)
        if isinstance(thing, Agent):
            thing.performance = 0
            self.agents.append(thing)

    def step(self):
        """Ejecuta un paso del entorno (llama a los programas de los agentes, obtiene sus acciones y las ejecuta). """
        if not self.is_done():
            actions = []
            for agent in self.agents:
                if agent.alive:
                    actions.append(agent.program(self.percept(agent)))
                else:
                    actions.append("")
            for (agent, action) in zip(self.agents, actions):
                self.execute_action(agent, action)

    def run(self, steps=1000):
        """Ejecuta steps pasos en el entorno."""
        for step in range(steps):
            if self.is_done():
                return
            self.step()

#### Clase <b>VacuumEnvironment</b>

Esta clase implementa el entorno del aspirador de 6 posiciones: loc_A, loc_B, loc_C, loc_D, loc_E y loc_F. Cada una de estas posiciones puede tener el estado 'Dirty' o 'Clean'. Un agente en este entorno percibe su localizacion y el estado de la misma

In [ ]:
# Complete las posiciones reemplazando los guiones bajos por números.
# Considere el siguiente ambiente:

########
# A  B #
# c  D # 
# E  F #
########

loc_A, loc_B, loc_C, loc_D, loc_E, loc_F = (0, 0), (0, 1), (1, 0), (1, 1), (2, 0), (2, 1)

class VacuumEnvironment(Environment):

    def __init__(self):
        super().__init__()
        self.status = {loc_A: random.choice(['Clean', 'Dirty']),
                       loc_B: random.choice(['Clean', 'Dirty']),
                       loc_C: random.choice(['Clean', 'Dirty']),
                       loc_D: random.choice(['Clean', 'Dirty']),
                       loc_E: random.choice(['Clean', 'Dirty']),
                       loc_F: random.choice(['Clean', 'Dirty']),}

    def thing_classes(self):
        return [ReflexVacuumAgent]

    def percept(self, agent):
        """Retorna la posición del agente y el estado de la posición (Dirty/Clean)."""
        return (agent.location, self.status[agent.location])

    def execute_action(self, agent, action):
        """Implementa el MAPA DE TRANSICIÓN: Cambia la posición del agente y/o el estado de la posición; 
        Cada aspiración (acción 'suck') en una localización Dirty provoca un aumento de desempeño en 10 unidades;
        Cada movida efectiva Right, Left, Up o Down provoca una disminución de desempeño en 1 unidad """
        
        if action == 'Suck':
            self.status[agent.location] = 'Clean'
            agent.performance += 10
        else:
            x, y = agent.location

            if (action == 'Right') and (y == 0):
                y = 1
            elif (action == 'Left') and (y == 1):
                y = 0
            elif (action == 'Up') and (x > 0):
                x -= 1
            elif (action == 'Down') and (x < 2):
                x += 1
            
            agent.location = x, y
            agent.performance -= 1

    def default_location(self, thing):
        """Devuelve una posicion aleatoria."""
        return random.choice([loc_A, loc_B, loc_C, loc_D, loc_E, loc_F])

# Agente Aspirador de 6 posiciones con Programa Reactivo Simple

Este agente es el agente aspirador de cuatro posiciones que usa un programa reactivo simple: realiza una accion basado en la percepción (posicion, estado) actual

In [ ]:
def ReflexVacuumAgent():
    
    def program(percept):
        location, status = percept
        
        # TO DO: Crear el programa de agente para que sea racional a la medida de desempeño del entorno VacuumEnvironment
        # Pista: piensa bien las acciones que debe retornar el agente para maximizar su medida de desempeño!
        
            
        return action
    return Agent(program)

# Probando el agente reflexivo simple en su entorno

In [ ]:
"""Crea el entorno del aspirador de 6 posiciones con 2 posiciones en estado 'Dirty'"""
e = VacuumEnvironment()
e.status = {loc_A: 'Clean',  loc_B: 'Dirty', loc_C: 'Clean', loc_D: 'Dirty', loc_E: 'Clean', loc_F: 'Dirty'}

"""Crea un agente reflexivo simple"""
a = ReflexVacuumAgent()

"""Añade el agente creado al entorno en posicion loc_A"""
e.add_thing(a, location=loc_A) 

# Imprime el estado inicial del ambiente y localizacion del agente
print("Estado Inicial del Ambiente: {}".format(e.status))
print("ReflexVacuumAgent esta localizado en {} con desempeño = {}".format(a.location, a.performance))

"""Ejecuta el entorno 6 pasos y obtiene el desempeño del agente"""
e.run(6)

# Imprime el estado actual del ambiente, localizacion del agente y su desempeño
print("Estado del Ambiente despues de 6 pasos: {}".format(e.status))
print("ReflexVacuumAgent esta localizado en {} con desempeño = {}".format(a.location, a.performance))


# Preguntas:
    1. Describa qué comportamiento (acciones) está favoreciendo el entorno   (2 pts)
    2. Escriba el programa de agente ReflexVacuumAgent para que sea racional a la medida de desempeño del entorno (5 pts)
    3. Modifique la medida de desempeño que otorga el entorno para que se premie la suciedad efectivamente aspirada y se castigue la cantidad de movimientos realizados (5 pts)
    4. Reescriba el programa de agente ReflexVacuumAgent para que sea racional para la medida de desempeño modificada en 3. Ejecute 6 pasos del entorno. (5 pts)
    5.  Imagine que el agente solo pudiese percibir el estado de suciedad, mas no su ubicacion ¿Qué estructura de agente seria más adecuada para ese caso?.  Justifique (3 pts)